In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import itertools
import random
from IPython.display import display

from tqdm import tqdm

import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from tools.get_describe_record import get_describe_record

In [4]:
featuresA = [f"X{i}" for i in range(1, 84)]
featuresB = [f"X{i}" for i in range(1, 84)]

#featuresA = ["X30", "X41", "X28", "X33", "X76", "X31"]
#featuresB = ["X41", "X36", "X27", "X29", "X35", "X26"]

comb_A = list(itertools.combinations(featuresA, 4))
comb_B = list(itertools.combinations(featuresB, 4))

comb_AB = list(itertools.product(comb_A, comb_B))
random.shuffle(comb_AB)

best_rmse = 100
rmse_sp = 50
rmse_no = 100
rmse_all = 100
best_comb = None

# 異常スパイク検出用
batch_describe_df = pd.read_csv("../data/processed/batch_describe_df.csv")
test_batch = batch_describe_df.iloc[75:]


: 

: 

: 

In [ ]:

# 異常スパイク検知モデルと閾値スコアの読み込み
with open("../data/model/spike_detection_IFmodel.pkl", "rb") as f:
    spike_detection_model = pickle.load(f)
with open("../data/score/IF_train_score.pkl", "rb") as f:
    spike_detection_score = pickle.load(f)
thr = np.quantile(spike_detection_score, 0.90)

score_test  = -spike_detection_model.score_samples(test_batch)
pred_iso = (score_test >= thr).astype(int)
spike_batches = test_batch.index[pred_iso == 1].tolist()
normal_batches = test_batch.index[pred_iso == 0].tolist()
print(f"spike_batches: {spike_batches}")
print(f"normal_batches: {normal_batches}")


# 学習用データ
tagged_data = pd.read_csv("../data/processed/processe_tagged_anormaly.csv")
anomaly_df = tagged_data[tagged_data["is_anomaly"] == 1]
normal_df = tagged_data[tagged_data["is_anomaly"] == 0]
anomaly_train_df = anomaly_df.iloc[:75]
nomaly_train_df = normal_df.iloc[:75]

# テスト用データ
test_df = pd.read_csv("../data/processed/processe_tagged_anormaly.csv")
anomaly_test_df = test_df[test_df["batch_id"].isin(spike_batches)]
normaly_test_df  = test_df[test_df["batch_id"].isin(normal_batches)]
test_df_original = test_df[test_df["batch_id"] >=75]

In [1]:
# 特徴量エンジニアリング
def feature_engineering1(df, SENSOR_COLS, LAGS, WINS):
    base = df[SENSOR_COLS].copy()
    feats = {}  # ★ここに全部貯める

    for c in SENSOR_COLS:
        s = df[c]

        # lag
        for l in LAGS:
            feats[f"{c}_lag{l}"] = s.shift(l)

        # diff / pct
        feats[f"{c}_diff1"] = s.diff(1)
        feats[f"{c}_diff2"] = s.diff(2)
        feats[f"{c}_pct1"]  = s.pct_change(1).replace([np.inf, -np.inf], np.nan)

        # rolling
        for w in WINS:
            r = s.rolling(w, min_periods=1)
            rmean = r.mean()
            rstd  = r.std()
            feats[f"{c}_rmean{w}"] = rmean
            feats[f"{c}_rstd{w}"]  = rstd
            feats[f"{c}_rmax{w}"]  = r.max()
            feats[f"{c}_rmin{w}"]  = r.min()
            feats[f"{c}_z{w}"]     = (s - rmean) / (rstd + 1e-9)
            feats[f"{c}_dev{w}"]   = s - rmean
        
        feats[f"{c}_energy"] = (
            feats[f"{c}_rstd20"] * feats[f"{c}_rmax20"]
        )

        feats[f"{c}_jump"] = s - s.shift(5)

    feat_df = pd.concat([base, pd.DataFrame(feats, index=df.index)], axis=1)
    feat_df = feat_df.ffill().bfill().fillna(0)
    return feat_df

def df_set_datetime(df, col_name):
    for col in col_name:
        df[col] = pd.to_datetime(df[col])
    return df

# process_end_timeを用いて経過時間, ラグを取得する。
def get_elapsed_day(df, base_time=None):
    if base_time == None:
        base_time = df['process_end_time'].min()
    df['elapsed_day'] = (df['process_end_time'] - base_time).dt.days
    return df

def set_LagOV(df,target="OV", lag_record_num=1, window=3):
    df[f"{target}_lag{lag_record_num}"] = df[target].shift(lag_record_num)
    df[f"{target}_diff"] = df[target].diff(1).shift(1)  
    df[f"{target}_roll_mean{window}"] = df[target].rolling(window).mean().shift(1)
    df[f"{target}_roll_std{window}"] = df[target].rolling(window).std().shift(1)
    #df = df.dropna().reset_index(drop=True)
    return df

def feature_engineering2(df, normal_features):
    df = df.copy()
    df = df_set_datetime(df, ["process_end_time", "final_mes_time"])
    df = get_elapsed_day(df)
    df = set_LagOV(df)
    df = df.drop(columns=[col for col in df.columns if col not in normal_features])
    return df

In [ ]:
for (comb_A, comb_B) in comb_AB:
    # スパイクモデルの学習
    SENSOR_COLS = list(comb_A)
    LAGS = [1, 2, 3, 5, 10, 20]
    WINS = [3, 5, 10, 20]

    train_feature_df = feature_engineering1(anomaly_train_df, SENSOR_COLS=SENSOR_COLS, LAGS=LAGS, WINS=WINS)

    # モデルの学習
    X_sp = train_feature_df
    y_sp = np.log1p(anomaly_train_df["OV"].values)

    # 高OVを重くする（まずは80%点を基準）
    q = np.percentile(anomaly_train_df["OV"].values, 90)  # “頂点”の境界
    w = np.ones_like(anomaly_train_df["OV"].values, dtype=float)
    w[anomaly_train_df["OV"].values >= q] = 1 + ((anomaly_train_df["OV"].values[anomaly_train_df["OV"].values >= q] / (q+1e-9))**3)
    w = np.clip(w, 1.0, np.percentile(w, 95))

    spike_predict_model = RandomForestRegressor(
        n_estimators=600,
        max_depth=18,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    )
    spike_predict_model.fit(X_sp, y_sp, sample_weight=w)

    test_feature_df = feature_engineering1(anomaly_test_df, SENSOR_COLS=SENSOR_COLS, LAGS=LAGS, WINS=WINS)
    spiked_pred = spike_predict_model.predict(test_feature_df)
    spiked_pred = np.expm1(spiked_pred)
    spiked_pred_series = pd.Series(spiked_pred, index=anomaly_test_df.index)
    rmse_sp = np.sqrt(mean_squared_error(anomaly_test_df["OV"], spiked_pred))
    #print(spiked_pred)

    # フラット部分の学習
    normal_features = list(comb_B) + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3", "OV"]
    train_df = feature_engineering2(nomaly_train_df, normal_features)
    test_df = feature_engineering2(normaly_test_df, normal_features)

    pipeline = Pipeline([
        #("scaler", StandardScaler()),
        ("scaler", StandardScaler()),
        ("model", LGBMRegressor(
            objective="regression",
            metric="rmse",
            verbose=-1,
            random_state=42
        ))
    ])
    train_X = train_df[normal_features].drop(columns=["OV"])
    pipeline.fit(train_X, train_df["OV"])

    test_X = test_df[normal_features].drop(columns=["OV"])
    normal_pred = pipeline.predict(test_X)
    rmse_no = np.sqrt(mean_squared_error(normaly_test_df["OV"], normal_pred))
    normal_pred_series = pd.Series(normal_pred, index=normaly_test_df.index)
    df_eval = pd.concat([test_df_original["OV"].rename("y_true"), spiked_pred_series.rename("y_pred")], axis=1)
    df_eval["y_pred"] = df_eval["y_pred"].fillna((normal_pred_series))
    rmse_all = np.sqrt(mean_squared_error(df_eval["y_true"], df_eval["y_pred"]))
    
    print(f"COMB:{comb_A, comb_B} \t RMSE (ALL): {rmse_all:.3f} \t RMSE (SP): {rmse_sp:.3f} \t RMSE (NO): {rmse_no:.3f} \t RMSE BEST: {best_rmse:.3f}")
    if rmse_all < best_rmse:
        print("BEST MODEL FOUND")
        best_rmse = rmse_all
        best_comb = (comb_A, comb_B)
print(f"best_comb: {best_comb} \t RMSE (ALL): {best_rmse:.3f}")